# Evaluate Printer Log Usage
This notebook is just an automated way to report on 3D printer usage. It expects the latest data file to be downloaded locally, whatever the source.

## Check Environment
I usually run these kind of notebooks in isolated virtual environments, This next bit of code check to make sure that the Python kernel is in the location associated with that virtual environment (only you can tell.) and the Python version.

In [15]:
import sys, os
print("cwd:{}".format(os.getcwd()))
print("exe:{}".format(sys.executable))
print("ver:{}".format(sys.version))

cwd:/home/rgordon/projects/barn-printer-logs/notebooks
exe:/home/rgordon/projects/barn-printer-logs/pyenv/bin/python3
ver:3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


## Standard Imports
I'll put all of the libraries/packages that we use in the notebook in this section, so you determine quickly whether there's anything missing.

In [5]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

## Load and Prep the Data
Npw we need to load and prepare the data for downstream usage. We'll check it here and apply what's needed. I'd expect this area to evolve as data grows in complexity or requirements.

In [23]:
DATA_DIR = os.path.join("..","data")
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".tsv")]
print(files)

['printer-logs.tsv']


In [20]:
df_raw = pd.read_csv(os.path.join(DATA_DIR, files[0]), sep='\t')

In [39]:
print("Shape of loaded data is {}".format(df_raw.shape))
print(df_raw.columns)
print(df_raw.index)
# columns can have whitespace around them - kill it
df_raw.columns = df_raw.columns.str.strip()
df_raw.head()

Shape of loaded data is (20, 12)
Index(['date', 'name', 'filament', 'days', 'hours ', 'mins', 'z offset',
       'self test passed', 'belt x', 'belt y', 'usage hrs (calc) ', 'notes'],
      dtype='object')
RangeIndex(start=0, stop=20, step=1)


,date,name,filament,days,hours,mins,z offset,self test passed,belt x,belt y,usage hrs (calc),notes
0,2019-10-24,Prusa 01,5118.88,327.0,13.0,50.0,NaN,y,NaN,NaN,7861.8,NaN
1,2019-10-24,Prusa 02,5855.92,243.0,2.0,52.0,NaN,y,NaN,NaN,5834.9,NaN
2,2019-10-24,Prusa 03,3490.80,350.0,0.0,58.0,NaN,y,NaN,NaN,8401.0,NaN
3,2019-10-24,Prusa 05,1147.71,41.0,15.0,17.0,NaN,y,NaN,NaN,999.3,NaN
4,2019-11-07,Prusa 01,5122.33,327.0,21.0,6.0,-0.99,NaN,NaN,NaN,7869.1,NaN


We will not need all of these columns at present, so let's extract the ones we do need as a separate copy that we can manipulate as needed.

In [41]:
df_printers = df_raw[['date','name','filament','days','hours','mins']]
df_printers.head()

,date,name,filament,days,hours,mins
0,2019-10-24,Prusa 01,5118.88,327.0,13.0,50.0
1,2019-10-24,Prusa 02,5855.92,243.0,2.0,52.0
2,2019-10-24,Prusa 03,3490.80,350.0,0.0,58.0
3,2019-10-24,Prusa 05,1147.71,41.0,15.0,17.0
4,2019-11-07,Prusa 01,5122.33,327.0,21.0,6.0
